In [17]:
import json, os
import ccd
import numpy as np
import pandas as pd
from datetime import datetime
from test.shared import snap, rainbow

In [2]:
# detect results for x, y: -2094435, 1681005 took 2.455234 seconds to generate  VERIFIED 2.6 seconds locally
# detect results for x, y: -2033595, 1685955 took 1.879693 seconds to generate
# detect results for x, y: -2098875, 1694895 took 0.986887 seconds to generate  VERIFIED 1.8 seconds locally
pixl_x, pixl_y = -2094435, 1681005
pixl_x, pixl_y = -2098875, 1694895

inputs_url = "https://lcmaphost.com?x={x}&y={y}&acquired=1982-01-01/2015-12-31\
&ubid=LANDSAT_4/TM/SRB1&ubid=LANDSAT_4/TM/SRB2&ubid=LANDSAT_4/TM/SRB3&ubid=LANDSAT_4/TM/SRB4\
&ubid=LANDSAT_4/TM/SRB5&ubid=LANDSAT_4/TM/BTB6&ubid=LANDSAT_4/TM/SRB7&ubid=LANDSAT_4/TM/PIXELQA\
&ubid=LANDSAT_5/TM/SRB1&ubid=LANDSAT_5/TM/SRB2&ubid=LANDSAT_5/TM/SRB3&ubid=LANDSAT_5/TM/SRB4\
&ubid=LANDSAT_5/TM/SRB5&ubid=LANDSAT_5/TM/BTB6&ubid=LANDSAT_5/TM/SRB7&ubid=LANDSAT_5/TM/PIXELQA\
&ubid=LANDSAT_7/ETM/SRB1&ubid=LANDSAT_7/ETM/SRB2&ubid=LANDSAT_7/ETM/SRB3&ubid=LANDSAT_7/ETM/SRB4\
&ubid=LANDSAT_7/ETM/SRB5&ubid=LANDSAT_7/ETM/BTB6&ubid=LANDSAT_7/ETM/SRB7&ubid=LANDSAT_7/ETM/PIXELQA\
&ubid=LANDSAT_8/OLI_TIRS/SRB2&ubid=LANDSAT_8/OLI_TIRS/SRB3&ubid=LANDSAT_8/OLI_TIRS/SRB4&ubid=LANDSAT_8/OLI_TIRS/SRB5\
&ubid=LANDSAT_8/OLI_TIRS/SRB6&ubid=LANDSAT_8/OLI_TIRS/SRB7&ubid=LANDSAT_8/OLI_TIRS/BTB10\
&ubid=LANDSAT_8/OLI_TIRS/PIXELQA".format(chips_host=chips_host, x=pixl_x, y=pixl_y)

dates           = [i.split('=')[1] for i in inputs_url.split('&') if 'acquired=' in i][0]
chips_url       = inputs_url.split('?')[0]
specs_url       = chips_url.replace('/chips', '/chip-specs')
querystr_list   = inputs_url.split('?')[1].split('&')
requested_ubids = [i.replace('ubid=', '') for i in querystr_list if 'ubid=' in i]

In [3]:
# get chip id
chip_x, chip_y = snap(pixl_x, pixl_y)
xindex = int((pixl_x - chip_x) / 30)
yindex = int((chip_y - pixl_y) / 30)

In [7]:
print(xindex)
print(yindex)

57
97


In [5]:
rbow = rainbow(-2094585, 1682805, dates, specs_url, chips_url, requested_ubids)

In [22]:
def dtstr_to_ordinal(dtstr, iso=True):
    """ Return ordinal from string formatted date"""
    _fmt = '%Y-%m-%dT%H:%M:%SZ' if iso else '%Y-%m-%d %H:%M:%S'
    _dt = datetime.strptime(dtstr, _fmt)
    return _dt.toordinal()

def detect(rainbow, x, y):
    """ Return results of ccd.detect for a given stack of data at a particular x and y """
    try:
        # Beware: rainbow contains stacks of row-major two-dimensional arrays
        # for each band of data. These variables are used to make the order
        # of access clear.
        _start = datetime.now()
        row, col = y, x
        rainbow_date_array = np.array(rainbow['t'].values)
        
        result =  ccd.detect([dtstr_to_ordinal(str(pd.to_datetime(i)), False) for i in rainbow_date_array],
                             np.array(rainbow['blue'].values[:, row, col]),
                             np.array(rainbow['green'].values[:, row, col]),
                             np.array(rainbow['red'].values[:, row, col]),
                             np.array(rainbow['nir'].values[:, row, col]),
                             np.array(rainbow['swir1'].values[:, row, col]),
                             np.array(rainbow['swir2'].values[:, row, col]),
                             np.array(rainbow['thermal'].values[:, row, col]),
                             np.array(rainbow['cfmask'].values[:, row, col], dtype=int),
                             params={})
        print("detect took {} seconds to run".format((datetime.now()-_start).total_seconds()))
        return result
    except Exception as e:
        raise Exception(e)

In [23]:
res = detect(rbow, xindex, yindex)

detect took 1.692778 seconds to run


/home/caustin/miniconda2/envs/ccd35/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [24]:
res

{'algorithm': 'lcmap-pyccd:2017.06.20',
 'change_models': [ChangeModel(start_day=724232, end_day=725312, break_day=725392, observation_count=24, change_probability=0, curve_qa=14, blue=SpectralModel(rmse=317.08029884920501, coefficients=array([  1.38080991e-01,  -8.54652283e+01,  -1.54894351e+02,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00]), intercept=-99138.917285486939, magnitude=0.0), green=SpectralModel(rmse=346.14336370094196, coefficients=array([  2.19954445e-02,  -1.42550712e+02,  -2.17054211e+02,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00]), intercept=-14593.816775893074, magnitude=0.0), red=SpectralModel(rmse=473.49573485301141, coefficients=array([  1.44378634e-01,  -1.78636744e+02,  -3.13199187e+02,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00]), intercept=-103317.92807450811, magnitude=0.0), nir=SpectralModel(rmse=887.45513171745847, coefficients=array

In [27]:
print(dates)

1982-01-01/2015-12-31
